In [2]:
import requests
import pandas as pd
import datetime

In [80]:
#returns a list of dicts which are meant to be transformed to a dataframe
def get_qualifying_results(season,round):
    
    qualifying_url= 'http://ergast.com/api/f1/'+season+'/'+round+'/qualifying.json'
    r = requests.get(qualifying_url)
    json=r.json()
    MRData=json["MRData"]
    RaceTable = MRData["RaceTable"]
    try:
        race = RaceTable["Races"][0]
    except:
        return []

    #Qualifying Round Details
    round_season    = race["season"]
    round_round     = race["round"]
    round_circuit   = race["Circuit"]["circuitName"]
    round_date      = race["date"]

    #Qualifying Results
    qualifying_results = race["QualifyingResults"]
    #print("Qualifying: ")
    #print(qualifying_results[0].keys())

    result_list = []
    for q_result in range(len(qualifying_results)):
        #Driver Details
        driver = qualifying_results[q_result]["Driver"]
        driver_name = driver["givenName"] + " " + driver["familyName"]
        driver_nationality = driver["nationality"]
        driver_code = driver["code"]
        #Constructor Details
        constructor = qualifying_results[q_result]["Constructor"]
        constructor_name = constructor["name"]

        #Qualifying Details
        Q1 = qualifying_results[q_result]["Q1"]
        try:
            Q2 = qualifying_results[q_result]["Q2"]
        except:
            Q2 = None
        try:
            Q3 = qualifying_results[q_result]["Q3"]
        except:
            Q3 = None
        position = qualifying_results[q_result]["position"]
        result_dict = {"season": round_season,
                        "round": round_round,
                        "circuit": round_circuit,
                        "date": round_date,
                        "driver_name": driver_name,
                        "driver_code": driver_code,
                        "driver_nationality": driver_nationality,
                        "constructor_name": constructor_name,
                        "Q1": Q1,
                        "Q2": Q2,
                        "Q3": Q3,
                        "position": position}
        result_list.append(result_dict)
    return result_list

season ="2023"
round = "9"
qualifying_url= 'http://ergast.com/api/f1/'+season+'/'+round+'/qualifying.json'
r = requests.get(qualifying_url)
json=r.json()
MRData=json["MRData"]
RaceTable = MRData["RaceTable"]
race = RaceTable["Races"][0]
qualifying_results = race["QualifyingResults"]


In [82]:
first_year = 2010
last_year = 2023
qualifying_list=[]


for season in range(first_year,last_year+1):
    season=str(season)
    #get num races
    qualifying_url= 'http://ergast.com/api/f1/'+season+'/1/qualifying.json'
    r = requests.get(qualifying_url)
    num_races=r.json()["MRData"]["total"]
    print("getting data for season "+season+ ". There are : "+num_races+ " number of races.")

    for round in range(int(num_races)):
        round=str(round+1)
        #print("getting data for season "+season+ " and round: "+round)
        qualifying_list=qualifying_list+get_qualifying_results(season=season,round=round)


getting data for season 2010. There are : 24 number of races.
getting data for season 2011. There are : 24 number of races.
getting data for season 2012. There are : 24 number of races.
getting data for season 2013. There are : 22 number of races.
getting data for season 2014. There are : 22 number of races.
getting data for season 2015. There are : 18 number of races.
getting data for season 2016. There are : 22 number of races.
getting data for season 2017. There are : 20 number of races.
getting data for season 2018. There are : 20 number of races.
getting data for season 2019. There are : 20 number of races.
getting data for season 2020. There are : 20 number of races.
getting data for season 2021. There are : 20 number of races.
getting data for season 2022. There are : 20 number of races.
getting data for season 2023. There are : 20 number of races.


In [83]:
#make list into df
df = pd.DataFrame(qualifying_list)
df["ReadInTime"]=datetime.datetime.now()

In [86]:
#read df into sql server database
database="F1"
from sqlalchemy import create_engine
engine = create_engine('mssql+pyodbc://localhost/'+database+'?driver=SQL+Server+Native+Client+11.0')
df.to_sql('stg_qualifying', engine, if_exists="replace", index=False)

-1

#run proc in sql server
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                        'Server=localhost;'
                        'Database=SMHI;'
                        'Trusted_Connection=yes;')

cursor = conn.cursor()
cursor.execute('Clean_Väder_DateTime')
cursor.execute('Clean_Väder_Date')
conn.commit()
cursor.close()
conn.close()
